<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/RTAS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall opencv-python

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30


In [2]:
!pip uninstall opencv-contrib-python

Found existing installation: opencv-contrib-python 4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-contrib-python-4.1.2.30


In [3]:
!pip install opencv-contrib-python

     |████████████████████████████████| 66.6 MB 120 kB/s 


In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylab import *
from numpy import *
from scipy import linalg

In [ ]:
k = [[1698.873755, 0.000000, 971.7497705],
     [0.000000, 1698.8796645, 647.7488275],
     [0.000000, 0.000000, 1.000000]]

k = np.asarray(k)

In [ ]:
k

array([[1.69887376e+03, 0.00000000e+00, 9.71749770e+02],
       [0.00000000e+00, 1.69887966e+03, 6.47748827e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
k_inv = np.linalg.inv(k)

In [ ]:
k_inv

array([[ 5.88625257e-04,  0.00000000e+00, -5.71996458e-01],
       [ 0.00000000e+00,  5.88623209e-04, -3.81279994e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [ ]:
img1 = cv2.imread('/content/im1.jpg', 0)
img2 = cv2.imread('/content/im2.jpg', 0)

In [ ]:
def find_correspondences(img1, img2):
  sift = cv2.xfeatures2d.SIFT_create()
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5),dict(checks=50))
  matches = flann.knnMatch(des1,des2,k=2)
  pts1 = []
  pts2 = []

  # ratio test as per Lowe's paper
  j = 0
  for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        pts2.append((kp2[m.trainIdx].pt, 1))
        pts1.append(kp1[m.queryIdx].pt)
        j = j+1
        if j == 100:
          break
  pts1 = [list(tup)+[1] for tup in pts1]
  pts2 = [list(tup)+[1] for tup in pts2]
  pts1, pts2 = np.asarray(pts1), np.asarray(pts2)
  pts1 = np.float64(pts1)
  pts2 = np.float64(pts1)

  return pts1, pts2

In [ ]:
pts1, pts2 = find_correspondences(img1, img2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
def convert_to_camera_coordinate_system(pts1, pts2, k):
  k_inv = np.linalg.inv(k)
  pts1 = [np.dot(k_inv, item) for item in pts1]
  pts2 = [np.dot(k_inv, item) for item in pts2]
  return pts1, pts2

In [ ]:
def compute_essential(pts1, pts2):
  
  x1, x2 = convert_to_camera_coordinate_system(pts1, pts2, k)
  x1 = np.asarray(x1)
  x2 = np.asarray(x2)
  n = x1.shape[1]
  if x2.shape[1] != n:
    raise ValueError("Number of points don't match.")
 
  # build matrix for equations
  A = []
  for i in range(8):
      z = np.kron(pts2[i], pts1[i])
      A.append(z)
  A = np.asarray(A)
  #print(A.shape)

  U,S,V = linalg.svd(A)
  E = V[-1].reshape(3,3)
  # constrain F
  # make rank 2 by zeroing out last singular value
  Ua,Sa,Va = linalg.svd(E)
  
  sigma = (Sa[0]+Sa[1])/2
  Sa[0] = sigma
  Sa[1] = sigma
  Sa[2] = 0
  
  Ea = np.dot(Ua,np.dot(diag(Sa),Va))
  

  return sigma , Ea


In [ ]:
sigma, E = compute_essential(pts1, pts2)

In [ ]:
E

array([[-3.43233757e-22,  1.80399143e-08, -3.09152220e-06],
       [-1.80399142e-08, -1.65426508e-18,  7.07106781e-01],
       [ 3.09152221e-06, -7.07106781e-01, -7.18875573e-20]])

In [ ]:
np.linalg.det(E)

5.40243782500198e-30

In [ ]:
sigma

0.7071067811865477

In [ ]:
U,S,V = linalg.svd(E)

In [ ]:
print(U)

[[ 4.37214707e-06  2.22044605e-16 -1.00000000e+00]
 [-9.99982975e-01 -5.83518599e-03 -4.37207263e-06]
 [-5.83518599e-03  9.99982975e-01 -2.55122913e-08]]


In [ ]:
print(-U[:,2])

[1.00000000e+00 4.37207263e-06 2.55122913e-08]


In [ ]:
def compute_R_and_T(E, sigma):

  U,S,V = linalg.svd(E)

  R_z = [[0, -1, 0],
         [1, 0, 0],
         [0, 0, 1]]
  R_z = np.asarray(R_z)
             
  R1 = np.dot(U ,np.dot(R_z, V))
  R2 = np.dot(U ,np.dot(np.transpose(R_z), V))
  t1 = U[:,2]
  t2 = -U[:,2]

  return R1, R2, t1, t2

In [ ]:
R1, R2, t1, t2 = compute_R_and_T(E, sigma)

In [ ]:
def compute_P_from_essential(E):
    """    Computes the second camera matrix (assuming P1 = [I 0]) 
        from an essential matrix. Output is a list of four 
        possible camera matrices. """
    
    U,S,V = svd(E)   
    
    # create matrices (Hartley p 258)
    #Z = skew([0,0,-1])
    W = array([[0,-1,0],[1,0,0],[0,0,1]])
    
    # return all four solutions
    P2 = [vstack((dot(U,dot(W,V)).T,U[:,2])).T,
             vstack((dot(U,dot(W,V)).T,-U[:,2])).T,
            vstack((dot(U,dot(W.T,V)).T,U[:,2])).T,
            vstack((dot(U,dot(W.T,V)).T,-U[:,2])).T]

    return np.asarray(P2)


In [ ]:
N = compute_P_from_essential(E)

In [ ]:
N.shape

(4, 3, 4)

In [ ]:
I_O = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]
M = np.dot(k, I_O)

In [ ]:
M

array([[1.69887376e+03, 0.00000000e+00, 9.71749770e+02, 0.00000000e+00],
       [0.00000000e+00, 1.69887966e+03, 6.47748827e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [ ]:
R1.shape

(3, 3)

In [ ]:
t1 = np.expand_dims(t1, axis = 1)

In [ ]:
t1.shape

(3, 1)

In [ ]:
R_t = np.hstack((R1, t1))
N = np.dot(k, R_t)

In [ ]:
R_t

array([[-1.00000000e+00, -4.93387383e-16, -2.24927448e-16,
        -1.00000000e+00],
       [ 4.94700143e-16, -1.00000000e+00, -3.23779540e-18,
        -4.37207263e-06],
       [ 3.12591306e-17,  2.18596596e-18, -1.00000000e+00,
        -2.55122913e-08]])

In [ ]:
N.shape

(3, 4)

In [ ]:
M.shape

(3, 4)

In [ ]:
A = np.vstack((M, N))

In [ ]:
A.shape

(6, 4)